In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("ratings.csv")
df = df[df["book_id"] < 300]

In [3]:
user_book_matrix = df.pivot_table(index='user_id', columns='book_id', values='rating')
user_book_matrix.head()

book_id,1,2,3,4,5,6,7,8,9,10,...,290,291,292,293,294,295,296,297,298,299
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,5.0,NaN,NaN,5.0,NaN,NaN,4.0,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,5.0,NaN,4.0,4.0,NaN,4.0,4.0,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

user_book_matrix = user_book_matrix.fillna(0)

book_similarity = cosine_similarity(user_book_matrix.T)
book_similarity_df = pd.DataFrame(book_similarity, index=user_book_matrix.columns, columns=user_book_matrix.columns)

book_similarity_df.head()


book_id,1,2,3,4,5,6,7,8,9,10,...,290,291,292,293,294,295,296,297,298,299
book_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.589840,0.561142,0.448696,0.396913,0.426246,0.400916,0.338818,0.401081,0.369558,...,0.190419,0.192924,0.205592,0.135734,0.219378,0.194561,0.162346,0.124495,0.144393,0.149826
2,0.589840,1.000000,0.523809,0.493046,0.441769,0.322296,0.525491,0.399516,0.440514,0.429215,...,0.150590,0.123093,0.246578,0.176630,0.199411,0.144276,0.186081,0.151050,0.168024,0.093427
3,0.561142,0.523809,1.000000,0.362559,0.309284,0.313530,0.310542,0.265028,0.364219,0.364369,...,0.114015,0.109889,0.159950,0.100927,0.148670,0.101360,0.151273,0.052915,0.084239,0.128472
4,0.448696,0.493046,0.362559,1.000000,0.587586,0.282096,0.425565,0.551808,0.370716,0.479950,...,0.140764,0.197340,0.184202,0.208351,0.096415,0.150706,0.167405,0.199934,0.167066,0.112236
5,0.396913,0.441769,0.309284,0.587586,1.000000,0.257952,0.396766,0.577414,0.331094,0.466008,...,0.151026,0.170049,0.159908,0.197671,0.072222,0.132243,0.147494,0.222912,0.166270,0.084218


In [5]:
def recommend_books_from_list(books_list, user_book_matrix, book_similarity_df, top_n=5):
    books_in_matrix = [book for book in books_list if book in user_book_matrix.columns]

    if not books_in_matrix:
        print("Brak książek w macierzy danych!")
        return []

    similar_scores = book_similarity_df[books_in_matrix].mean(axis=1)
    similar_scores = similar_scores.drop(books_in_matrix)
    recommended_books = similar_scores.nlargest(top_n)

    return recommended_books.index.tolist()

books_list = [18, 21, 23,24] 
recommended_books = recommend_books_from_list(books_list, user_book_matrix, book_similarity_df, top_n=5)
print("Polecane książki:", recommended_books)


Polecane książki: [27, 25, 2, 17, 20]


In [6]:
book_similarity_df.columns=book_similarity_df.columns.astype(str)

In [7]:
book_similarity_df = book_similarity_df.round(decimals=3)
book_similarity_df

book_id,1,2,3,4,5,6,7,8,9,10,...,290,291,292,293,294,295,296,297,298,299
book_id,,,,,,,,,,,,,,,,,,,,,
1,1.000,0.590,0.561,0.449,0.397,0.426,0.401,0.339,0.401,0.370,...,0.190,0.193,0.206,0.136,0.219,0.195,0.162,0.124,0.144,0.150
2,0.590,1.000,0.524,0.493,0.442,0.322,0.525,0.400,0.441,0.429,...,0.151,0.123,0.247,0.177,0.199,0.144,0.186,0.151,0.168,0.093
3,0.561,0.524,1.000,0.363,0.309,0.314,0.311,0.265,0.364,0.364,...,0.114,0.110,0.160,0.101,0.149,0.101,0.151,0.053,0.084,0.128
4,0.449,0.493,0.363,1.000,0.588,0.282,0.426,0.552,0.371,0.480,...,0.141,0.197,0.184,0.208,0.096,0.151,0.167,0.200,0.167,0.112
5,0.397,0.442,0.309,0.588,1.000,0.258,0.397,0.577,0.331,0.466,...,0.151,0.170,0.160,0.198,0.072,0.132,0.147,0.223,0.166,0.084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,0.195,0.144,0.101,0.151,0.132,0.139,0.128,0.126,0.159,0.083,...,0.080,0.110,0.045,0.053,0.026,1.000,0.025,0.067,0.076,0.062
296,0.162,0.186,0.151,0.167,0.147,0.076,0.143,0.128,0.121,0.134,...,0.045,0.037,0.102,0.103,0.036,0.025,1.000,0.041,0.063,0.055
297,0.124,0.151,0.053,0.200,0.223,0.057,0.199,0.252,0.106,0.123,...,0.038,0.045,0.070,0.094,0.013,0.067,0.041,1.000,0.148,0.013


In [8]:
book_similarity_df.to_csv('book_similarity_matrix.csv')

In [9]:
!pip install fastparquet